In [ ]:
from reportlab.pdfgen import canvas
from reportlab.lib.pagesizes import letter
import PyPDF2
import os
import pickle
import pandas as pd
import spacy
import re
from collections import defaultdict
from time import time
import numpy as np
from sklearn import metrics
from time import time
import numpy as np
from sklearn import metrics
from reportlab.lib.pagesizes import letter
from reportlab.pdfgen import canvas
from reportlab.lib import colors

# Step 3: Data Processing to PDF

In this example, I wanted to narrow the focus to look at 10-K disclosures from 2023 - Utility Sector

In [ ]:
# Load the serialized data from the pickle file
with open('Cleaned_US_Item1_1A.pkl', 'rb') as f:
    documents_info = pickle.load(f)

# Create a DataFrame from the documents_info list
Cleaned_US_Item1_1A = pd.DataFrame(documents_info)


In [ ]:
def preprocess_text(text):
    return ' '.join(text.split())

In [ ]:
unique_utility_df = df_2023[df_2023['sector'] == 'Utilities']

In [ ]:
def draw_metadata(c, metadata, width, height):
    """Draws metadata at the top of each page."""
    c.setFont("Helvetica", 12)
    c.drawString(72, height - 50, f"Ticker: {metadata['ticker']}, Sector: {metadata['sector']}, Filed At: {metadata['filedAt']}")

def add_text_to_page(c, text, metadata, width, height):
    """Adds text to a page ensuring metadata is drawn first and proper spacing is maintained."""
    draw_metadata(c, metadata, width, height)
    text_object = c.beginText(72, height - 100)  # Adjusted to leave space below the metadata
    text_object.setFont("Helvetica", 10)
    for line in text.split():
        line = preprocess_text(line)  # Preprocess each line if necessary
        if text_object.getX() + c.stringWidth(line) > width - 72:
            text_object.textLine()  # Move to next line if text exceeds the page width
        if text_object.getY() < 100:  # Check if we're near the bottom of the page
            c.drawText(text_object)  # Draw the text collected so far
            c.showPage()  # Start a new page
            draw_metadata(c, metadata, width, height)  # Redraw metadata at top of the new page
            text_object = c.beginText(72, height - 100)
        text_object.textOut(line + " ")  # Add space between words
    c.drawText(text_object)  # Make sure to draw any remaining text
    c.showPage()  # Ensure a new page is started after finishing each company's text

def create_pdf(df, filename):
    """Creates a PDF file with each entry separated onto a new page with metadata at the top."""
    c = canvas.Canvas(filename, pagesize=letter)
    width, height = letter
    for index, row in df.iterrows():
        metadata = {'ticker': row['ticker'], 'sector': row['sector'], 'filedAt': row['filedAt']}
        add_text_to_page(c, row['text'], metadata, width, height)
    c.save()
    print(f"PDF saved as {filename}")

create_pdf(df=unique_utility_df, filename='utility.pdf')